# TUP stock performance

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import datetime as dt
import time
import os
from datawrapper import Datawrapper

In [3]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [4]:
dw_token = os.environ.get("dw_api")
dw = Datawrapper(access_token=dw_token)

#### Get date as a string and unix time

In [5]:
today = dt.datetime.today().strftime("%Y-%m-%d")

In [6]:
now = round(time.time())

---

#### Read data from Yahoo! Finance

In [7]:
src = pd.read_json(
    f"https://query1.finance.yahoo.com/v8/finance/chart/TUP?symbol=TSLA&period1=1341817200&period2={now}&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=IgiovSYKfWc&corsDomain=finance.yahoo.com"
)["chart"]["result"]

In [8]:
df = pd.DataFrame(columns=["datetime", "close"])

In [9]:
df["datetime"] = pd.to_datetime(src[0]["timestamp"], unit="s")
df["close"] = src[0]["indicators"]["quote"][0]["close"]

In [10]:
df["close"] = df["close"].round(2)

In [11]:
df["date"] = pd.to_datetime(df["datetime"].dt.date)

In [12]:
# latest_close = df.tail(1)["close"].round(2).iloc[0]
# latest_close

In [13]:
# jan_close = df.query('datetime > "2022-01-01"').head(1)["close"].round(2).iloc[0]

In [14]:
# df["jan_close"] = jan_close

In [15]:
df["symbol"] = "TUP"

---

## Export 

In [16]:
df[["date", "close"]].to_csv("data/processed/tupperware_closing_price.csv", index=False)

In [18]:
dw.add_data(chart_id="Y5mwn", data=df)

<Response [204]>

## Add competitor brand

In [ ]:
nwl = pd.read_json(
    f"https://query1.finance.yahoo.com/v8/finance/chart/NWL?symbol=TSLA&period1=1341817200&period2={now}&useYfid=true&interval=1d&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=IgiovSYKfWc&corsDomain=finance.yahoo.com"
)["chart"]["result"]

In [ ]:
nwl_df = pd.DataFrame(columns=["datetime", "close"])

In [ ]:
nwl_df["datetime"] = pd.to_datetime(nwl[0]["timestamp"], unit="s")
nwl_df["close"] = nwl[0]["indicators"]["quote"][0]["close"]

In [ ]:
nwl_df["close"] = nwl_df["close"].round(2)

In [ ]:
nwl_df["date"] = pd.to_datetime(nwl_df["datetime"].dt.date)

In [ ]:
# latest_close = df.tail(1)["close"].round(2).iloc[0]
# latest_close

In [ ]:
# jan_close = df.query('datetime > "2022-01-01"').head(1)["close"].round(2).iloc[0]

In [ ]:
# df["jan_close"] = jan_close

In [ ]:
nwl_df["symbol"] = "NWL"

### combine both to chart together

In [ ]:
tup_nwl = pd.concat([df, nwl_df]).reset_index(drop=True)[["date", "close", "symbol"]]

In [ ]:
len(tup_nwl)

In [ ]:
alt.Chart(tup_nwl[tup_nwl["date"] >= "2014-01-01"]).mark_line().encode(
    x=alt.X("date"), y=alt.Y("close"), color=alt.Color("symbol")
)